# Getting started with HTMD

Assuming that you have already downloaded and installed htmd, this tutorial guides you through the basic language features.

After importing, any object or function defined by htmd is available in the workspace.

In [1]:
from htmd import *

Videos from the HTMD2015 workshops are available on the Acellera youtube channel: https://www.youtube.com/user/acelleralive

You are on the latest HTMD version (1.0.11).


Let's look more carefully at the Molecule class features.

## Molecule objects

First create an empty molecule object by either,
* Fetching it from the Protein Data Bank, by using its PDB code:

In [2]:
mol = Molecule('3PTB')

or,
* just use your own PDB file:

In [3]:
# uncomment and change the filename
#mol = Molecule('yourprotein.pdb')

## Inspect your molecule

`Molecule.get` can be used to check and retrieve specific PDB fields. For example,
* Check the residue ids (numbers) of the triptophan residues present in your protein:

In [4]:
np.unique(mol.get('resid','resname TRP'))

array([ 51, 141, 215, 237])

* Store the coordinates of a specific atom:

In [5]:
mol.get('coords','resname TRP and resid 51 and name CA')

array([ 10.06900024,   3.77600002,  34.72100067], dtype=float32)

* Display the number of chains or segments present in your PDB file:

In [6]:
np.unique(mol.get('chain'))

array(['A'], dtype=object)

Note that, unless you select a specific atom, `np.unique()` is needed to avoid duplicates in the output.

## Duplicate/modify objects and fields

Use `Molecule.copy` to duplicate the molecule into a new object:

In [7]:
newmol = mol.copy()

`Molecule.write` can be used to output a PDB file of your whole molecule (or just a selection). The following command uses the above copied molecule `newmol` to write out a PDB file of the benzamidine ligand atoms present in the fetched PDB file, except for hydrogen atoms:

In [8]:
newmol.write('/tmp/ligand.pdb','resname BEN and noh')

Alternatively, `Molecule.filter` can be used to selecting specific parts of the molecule (e.g. chains, segments) and cleaning/removing all the rest. For example, clean all except for protein atoms in chain A:

In [9]:
mol.filter('chain A and protein')

2016-04-11 19:49:48,443 - htmd.molecule.molecule - INFO - Removed 72 atoms. 1629 atoms remaining in the molecule.


`Molecule.set` is instead used to change/name/rename specific fields of the molecule or a selection. For example, `Molecule.set` can create a segid called 'P' for all protein atoms:

In [10]:
mol.set('segid','P','protein')

or rename all HIS residues to 'HSN':

In [11]:
mol.set('resname','HSN','resname HIS')

## Joining molecules/segments

`Molecule.append` appends a Molecule object (e.g. ligand, water or ion segments, etc.) to another molecule object (e.g. protein, membrane). For example, to append the benzamidine ligand (saved above as a PDB file) to the molecule we are working with (which is now only the protein chain A), simply do:

In [12]:
ligand = Molecule('/tmp/ligand.pdb')
mol.append(ligand)

You can add an atom in a similar fashion.

In [13]:
atom = Molecule()
atom.record = 'ATOM'
atom.name = 'CA'
atom.resid = 0
atom.resname = 'XXX'
atom.chain = 'X'
atom.coords = [6, 3, 2]

newligand = ligand.copy()
newligand.insert(atom, 0)
newligand.resid

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Playing with coordinates

Coordinates can be used to perform geometric tasks on your molecule, such as translations. For example, calculate the geometric center of your molecule:

In [14]:
coo = mol.get('coords')
c = np.mean(coo,axis=0)

Then, `Molecule.moveBy` can be used to translate the molecule and center it on the origin [0, 0, 0] using the previosly calculated center `c`:

In [15]:
mol.moveBy(-c)

Finally, check the new center:

In [16]:
np.mean(mol.get('coords'),axis=0)

array([  2.15420940e-07,  -2.36497249e-06,   4.63504330e-06], dtype=float32)

Another common task is rotations (e.g. to build a protein - ligand system). For example, load up your ligand and calculate its geometric center using the commands mentioned above:

In [17]:
ligand = Molecule('/tmp/ligand.pdb')
ligcenter = np.mean(ligand.get('coords'),axis=0) # the commands above can

Use `Molecule.rotateBy` to rotate your ligand with the use of a rotation matrix `M`:

In [18]:
M = uniformRandomRotation()
ligand.rotateBy(M,center=ligcenter)

Note that the `uniformRandomRotation()` function provides the coordinates needed for uniformly distributed random rotation.